#### Copyright 2019 Autores de TensorFlow Hub.

Con licencia de Apache License, Versión 2.0 (la "Licencia");

In [0]:
# Copyright 2019 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Exploración de las incrustaciones giratorias TF-Hub CORD-19


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/hub/tutorials/cord_19_embeddings_keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/cord_19_embeddings_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/cord_19_embeddings_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/cord_19_embeddings_keras.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar cuaderno</a></td>
</table>

El módulo de incrustación de texto CORD-19 Swivel de TF-Hub (https://tfhub.dev/tensorflow/cord-19/swivel-128d/3) se creó para ayudar a los investigadores a analizar textos en lenguajes naturales relacionados con COVID-19. Estas incorporaciones se entrenaron en los títulos, autores, resúmenes, textos corporales y títulos de referencia de artículos en el [conjunto de datos CORD-19](https://pages.semanticscholar.org/coronavirus-research) .

En este colab vamos a:

- Analizar palabras semánticamente similares en el espacio de inserción
- Entrene un clasificador en el conjunto de datos de SciCite utilizando las incrustaciones de CORD-19


## Preparar


In [0]:
import functools
import itertools
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_hub as hub

from tqdm import trange

# Analizar las incrustaciones

Comencemos analizando la incrustación calculando y trazando una matriz de correlación entre diferentes términos. Si la incrustación aprendió a capturar con éxito el significado de diferentes palabras, los vectores de incrustación de palabras semánticamente similares deberían estar muy juntos. Echemos un vistazo a algunos términos relacionados con COVID-19.

In [0]:
# Use the inner product between two embedding vectors as the similarity measure
def plot_correlation(labels, features):
  corr = np.inner(features, features)
  corr /= np.max(corr)
  sns.heatmap(corr, xticklabels=labels, yticklabels=labels)

# Generate embeddings for some terms
queries = [
  # Related viruses
  'coronavirus', 'SARS', 'MERS',
  # Regions
  'Italy', 'Spain', 'Europe',
  # Symptoms
  'cough', 'fever', 'throat'
]

module = hub.load('https://tfhub.dev/tensorflow/cord-19/swivel-128d/3')
embeddings = module(queries)

plot_correlation(queries, embeddings)

Podemos ver que la incrustación capturó con éxito el significado de los diferentes términos. Cada palabra es similar a las otras palabras de su grupo (es decir, "coronavirus" tiene una alta correlación con "SARS" y "MERS"), mientras que son diferentes de los términos de otros grupos (es decir, la similitud entre "SARS" y "España" es cerca de 0).

Ahora veamos cómo podemos usar estas incrustaciones para resolver una tarea específica.

## SciCite: Clasificación por intención de citas

This section shows how one can use the embedding for downstream tasks such as text classification. We'll use the [SciCite dataset](https://www.tensorflow.org/datasets/catalog/scicite) from TensorFlow Datasets to classify citation intents in academic papers. Given a sentence with a citation from an academic paper, classify whether the main intent of the citation is as background information, use of methods, or comparing results.

In [0]:
builder = tfds.builder(name='scicite')
builder.download_and_prepare()
train_data, validation_data, test_data = builder.as_dataset(
    split=('train', 'validation', 'test'),
    as_supervised=True)

In [0]:
#@title Let's take a look at a few labeled examples from the training set
NUM_EXAMPLES =   10#@param {type:"integer"}

TEXT_FEATURE_NAME = builder.info.supervised_keys[0]
LABEL_NAME = builder.info.supervised_keys[1]

def label2str(numeric_label):
  m = builder.info.features[LABEL_NAME].names
  return m[numeric_label]

data = next(iter(train_data.batch(NUM_EXAMPLES)))


pd.DataFrame({
    TEXT_FEATURE_NAME: [ex.numpy().decode('utf8') for ex in data[0]],
    LABEL_NAME: [label2str(x) for x in data[1]]
})

## Entrenamiento de un clasificador de intenciones de citaton

We'll train a classifier on the [SciCite dataset](https://www.tensorflow.org/datasets/catalog/scicite) using Keras.  Let's build a model which use the CORD-19 embeddings with a classification layer on top.

In [0]:
#@title Hyperparameters { run: "auto" }

EMBEDDING = 'https://tfhub.dev/tensorflow/cord-19/swivel-128d/3'  #@param {type: "string"}
TRAINABLE_MODULE = False  #@param {type: "boolean"}

hub_layer = hub.KerasLayer(EMBEDDING, input_shape=[], 
                           dtype=tf.string, trainable=TRAINABLE_MODULE)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(3))
model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Entrenar y evaluar el modelo

Entrenemos y evaluemos el modelo para ver el desempeño en la tarea SciCite

In [0]:
EPOCHS =   35#@param {type: "integer"}
BATCH_SIZE = 32#@param {type: "integer"}

history = model.fit(train_data.shuffle(10000).batch(BATCH_SIZE),
                    epochs=EPOCHS,
                    validation_data=validation_data.batch(BATCH_SIZE),
                    verbose=1)

In [0]:
from matplotlib import pyplot as plt
def display_training_curves(training, validation, title, subplot):
  if subplot%10==1: # set up the subplots on the first call
    plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
    plt.tight_layout()
  ax = plt.subplot(subplot)
  ax.set_facecolor('#F8F8F8')
  ax.plot(training)
  ax.plot(validation)
  ax.set_title('model '+ title)
  ax.set_ylabel(title)
  ax.set_xlabel('epoch')
  ax.legend(['train', 'valid.'])

In [0]:
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 211)
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 212)

## Evaluar el modelo

Y veamos cómo funciona el modelo. Se devolverán dos valores. Pérdida (un número que representa nuestro error, los valores más bajos son mejores) y precisión.

In [0]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print('%s: %.3f' % (name, value))

Podemos ver que la pérdida disminuye rápidamente mientras que, especialmente, la precisión aumenta rápidamente. Tracemos algunos ejemplos para comprobar cómo se relaciona la predicción con las etiquetas verdaderas:

In [0]:
prediction_dataset = next(iter(test_data.batch(20)))

prediction_texts = [ex.numpy().decode('utf8') for ex in prediction_dataset[0]]
prediction_labels = [label2str(x) for x in prediction_dataset[1]]

predictions = [label2str(x) for x in model.predict_classes(prediction_texts)]


pd.DataFrame({
    TEXT_FEATURE_NAME: prediction_texts,
    LABEL_NAME: prediction_labels,
    'prediction': predictions
})

Podemos ver que para esta muestra aleatoria, el modelo predice la etiqueta correcta la mayoría de las veces, lo que indica que puede incorporar oraciones científicas bastante bien.

# ¿Que sigue?

Ahora que ha aprendido un poco más sobre las incrustaciones CORD-19 Swivel de TF-Hub, lo alentamos a participar en la competencia CORD-19 Kaggle para contribuir a obtener conocimientos científicos de los textos académicos relacionados con COVID-19.

- Participa en el [desafío CORD-19 Kaggle](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge)
- Obtenga más información sobre el [conjunto de datos de investigación abierta COVID-19 (CORD-19)](https://pages.semanticscholar.org/coronavirus-research)
- Consulte la documentación y más sobre las incrustaciones de TF-Hub en https://tfhub.dev/tensorflow/cord-19/swivel-128d/3
- Explore el espacio de incrustación CORD-19 con el [proyector de incrustación TensorFlow](http://projector.tensorflow.org/?config=https://storage.googleapis.com/tfhub-examples/tensorflow/cord-19/swivel-128d/3/tensorboard/projector_config.json)